# Third week notebook-part three

This notebook get the Postcode, Borough and Neighbourhood from Toronto and adds the longitude and latitude. Finally we do clustering and visualization.


First modules are imported



*Warning: This document uses both neighbo**u**rhood and neighborhood. This should be avoided* 

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
print('Modules imported :)')

Modules imported :)




Then the URI is defined

In [2]:
uri = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
print('The URI is: {} '.format(uri) )

The URI is: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 


Scarping the data and making the data frame

In [3]:
import pandas as pd
dfs = pd.read_html(uri, header=0)
df = dfs[0]

Using method chaining operations are carried out. 
 - First, the rows where Borough are assigned the value 'Not assigned' is removed
 - Then, for each postcode the name of the first borough and a comma separated list of the neighourhoods are compiled. If the boroughs have the same postcode this will only keep the name of the first borough
 - Finaly, reset index is used to make the postcode a colum and not an index

In [4]:
df1 = (df[df.Borough != 'Not assigned']
        .groupby('Postcode')
        .agg({'Borough': lambda x: x.iloc[0], 'Neighbourhood': lambda x: "%s" % ', '.join(x)})
        .reset_index()
      )

Making a copy to aviod mutating a data frame. Then, the values for the Neighbourhood which are not assigend are replaced with the name of the Borough. This assumes that no Borough have the value Not assigned.

In [5]:
df2 = df1.copy(deep=True)
df2.loc[df2.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df1.loc[df1.Neighbourhood == 'Not assigned', 'Borough']

In [6]:
import os
project_path = os.path.dirname(os.getcwd())
file_path = project_path + '/data/raw/Geospatial_Coordinates.csv'
file_path

'/Users/danielhaugstvedt/Developer/coursera_capstone/data/raw/Geospatial_Coordinates.csv'

In [7]:
postal_code_ll_df = (pd.read_csv(file_path)
                    .rename(columns={'Postal Code': 'Postcode'})
                    )

In [8]:
df3 = pd.merge(df2, postal_code_ll_df, how='left', on='Postcode')

## This is where the third week start

Import libraries used in the klustering

In [9]:
import numpy as np 
import json 

# Hard code the latitude and longitude instead
# from geopy.geocoders import Nominatim, 

import requests 
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


From the web i found that the latitude of North York, Toronto, ON, Canada is 43.761539, and the longitude is -79.411079
This method is preferd to avoid the use of Nominatim

In [10]:
latitude = 43.761539
longitude = -79.411079

In [11]:
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Use folium to create a map 

In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Rename instead of refactor
neighborhoods = df3.copy() 

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        popup=label).add_to(map_toronto)
    
map_toronto



The API secrets are not pushed to github. This is handled by placing them in a file named secrets and removing version control from that file.
Since the file is not in the same directory as the notebooks extra code is needed to add the path to the sys search path

In [13]:
import os
secrets_folder_path = os.path.dirname(os.getcwd())
secrets_folder_path
import sys
sys.path.insert(0, secrets_folder_path)


The secrets file containts two variables CLIENT_ID and CLIENT_SECRET

In [14]:
import secrets
CLIENT_ID = secrets.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = secrets.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    number = 0
    print('The number of names in the list is {}'.format(len(names)))
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        number += 1
        print('Nr.: {}, name: {}'.format(number, name))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
neighborhood_latitude_list = neighborhoods['Latitude'] # neighborhood latitude value
neighborhood_longitude_list = neighborhoods['Longitude'] # neighborhood longitude value
neighborhood_name_list = neighborhoods['Neighbourhood'] # neighborhood name
toronto_venues = getNearbyVenues(neighborhood_name_list, neighborhood_latitude_list, neighborhood_longitude_list)

The number of names in the list is 103
Nr.: 1, name: Rouge, Malvern
Nr.: 2, name: Highland Creek, Rouge Hill, Port Union
Nr.: 3, name: Guildwood, Morningside, West Hill
Nr.: 4, name: Woburn
Nr.: 5, name: Cedarbrae
Nr.: 6, name: Scarborough Village
Nr.: 7, name: East Birchmount Park, Ionview, Kennedy Park
Nr.: 8, name: Clairlea, Golden Mile, Oakridge
Nr.: 9, name: Cliffcrest, Cliffside, Scarborough Village West
Nr.: 10, name: Birch Cliff, Cliffside West
Nr.: 11, name: Dorset Park, Scarborough Town Centre, Wexford Heights
Nr.: 12, name: Maryvale, Wexford
Nr.: 13, name: Agincourt
Nr.: 14, name: Clarks Corners, Sullivan, Tam O'Shanter
Nr.: 15, name: Agincourt North, L'Amoreaux East, Milliken, Steeles East
Nr.: 16, name: L'Amoreaux West, Steeles West
Nr.: 17, name: Upper Rouge
Nr.: 18, name: Hillcrest Village
Nr.: 19, name: Fairview, Henry Farm, Oriole
Nr.: 20, name: Bayview Village
Nr.: 21, name: Silver Hills, York Mills
Nr.: 22, name: Newtonbrook, Willowdale
Nr.: 23, name: Willowdale Sout

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(1311, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhoods.'.format(len(toronto_venues['Neighborhood'].unique())))
print('The length of the neighborhood_name_list is {}'.format(len(neighborhood_name_list)))
print('The length of uniqe values in the neighborhood_name_list is {}'.format(len(neighborhood_name_list.unique())))

There are 238 uniques categories.
There are 100 uniques Neighborhoods.
The length of the neighborhood_name_list is 103
The length of uniqe values in the neighborhood_name_list is 103


It is likely that thee of the neighbourhoods does not have any venues. **I'm taking out the neighbourhoods that does not have any venues**

## Identify the naming problem 

Is there a category for venues called Neighborhood?

In [19]:
sum(toronto_venues['Venue Category'] == 'Neighborhood')

4

In [20]:
toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
253,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
364,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
690,"Adelaide, King, Richmond",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
707,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


The solution is to rename Neighborhood to Neighbourhood

## Prepare the data for machine learning

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe, look at name ...
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot_fixed = toronto_onehot[fixed_columns]
print(toronto_onehot_fixed.shape)
toronto_onehot_fixed.head()

(1311, 239)


,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Time for some machine learning

The suggested methods might not be the best. I'm going to try using the sum instead of the mean. This will classify neighborhoods with apox the same numbers of venues of the same type together. Larger areas will usually have higher number of venues. The best would be to divide by the area of the neighborhood.

In [22]:
toronto_sum_df = toronto_onehot_fixed.groupby('Neighbourhood').sum().reset_index()

In [23]:
print(toronto_sum_df.shape)

(100, 239)


In [24]:
# set number of clusters
kclusters = 5

# Sum the number of venues in each venue category
toronto_grouped_clustering = toronto_sum_df.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [25]:
# Merge labels with the labels
merged_df_tmp = pd.DataFrame({"Neighbourhood" : toronto_sum_df.loc[:,'Neighbourhood'], "Cluster Labels" : kmeans.labels_})
toronto_data = neighborhoods.loc[:, ['Neighbourhood', 'Latitude', 'Longitude']]
merged_df = pd.merge(merged_df_tmp, toronto_data, how = 'left', on = 'Neighbourhood')

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighbourhood'], merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

